In [1]:
import sys

# setting path
sys.path.append('..')

In [2]:
# local package
import classified

In [3]:
# outside package
import requests
import pandas as pd
import time
import json
from multiprocessing.dummy import Pool

In [4]:
def use_api(address, params=None, method="get"):
    if params is None:
        params = {"api_key": classified.API_KEY}
    else:
        params["api_key"] = classified.API_KEY
    method = getattr(requests, method)
    return method(address, params=params).json()

In [5]:
demo1 = use_api("https://api.opendota.com/api/proMatches")
demo1 = pd.DataFrame(demo1)

In [6]:
demo1.head()

,match_id,duration,start_time,radiant_team_id,radiant_name,dire_team_id,dire_name,leagueid,league_name,series_id,series_type,radiant_score,dire_score,radiant_win
0,6985360985,1526,1674580558,8254400.0,beastcoast,7119077.0,Alliance.LATAM,14886,DPC 2023 SA Winter Tour Division I – presented...,743689,1,7,33,False
1,6985359017,938,1674579927,8740972.0,ALPHA,8686786.0,Ravens,14783,EPL World Series: America 2022-2023 Season,743688,1,2,31,False
2,6985357123,1430,1674579683,8598633.0,YNT,8894818.0,Ooredoo Thunders,14691,European Pro League 2022-2023 Season,743668,1,9,30,False
3,6985354987,2081,1674579786,8849990.0,REAL ECLIPSE,8936613.0,Wild Dragon,14933,Dota 2 Space League,743671,1,22,42,False
4,6985329471,2908,1674578519,8970102.0,Amanita,8970163.0,Pull Up,14959,ESportsBattle | DOTA2 WC1,743650,1,34,51,False


In [7]:
demo2 = use_api("https://api.opendota.com/api/matches/{}".format(6897941675))

In [8]:
demo2

{'match_id': 6897941675,
 'barracks_status_dire': 51,
 'barracks_status_radiant': 63,
 'chat': [{'time': -54,
   'type': 'chatwheel',
   'key': '581',
   'slot': 9,
   'player_slot': 132},
  {'time': -51,
   'type': 'chatwheel',
   'key': '581',
   'slot': 9,
   'player_slot': 132},
  {'time': -36,
   'type': 'chatwheel',
   'key': '583',
   'slot': 6,
   'player_slot': 129},
  {'time': 8, 'type': 'chatwheel', 'key': '7', 'slot': 1, 'player_slot': 1},
  {'time': 9, 'type': 'chatwheel', 'key': '11', 'slot': 1, 'player_slot': 1},
  {'time': 16, 'type': 'chatwheel', 'key': '7', 'slot': 1, 'player_slot': 1},
  {'time': 165, 'type': 'chatwheel', 'key': '7', 'slot': 1, 'player_slot': 1},
  {'time': 292, 'type': 'chatwheel', 'key': '7', 'slot': 1, 'player_slot': 1},
  {'time': 292, 'type': 'chatwheel', 'key': '7', 'slot': 1, 'player_slot': 1},
  {'time': 440,
   'type': 'chatwheel',
   'key': '581',
   'slot': 9,
   'player_slot': 132},
  {'time': 445, 'type': 'chatwheel', 'key': '7', 'slot':

In [9]:
demo3 = use_api("https://api.opendota.com/api/heroes/1/matches")

In [10]:
demo3

[{'match_id': 6983772475,
  'start_time': 1674486703,
  'duration': 1231,
  'radiant_win': False,
  'leagueid': 14845,
  'league_name': 'Neon League',
  'radiant': True,
  'player_slot': 128,
  'account_id': 1041276674,
  'kills': 7,
  'deaths': 1,
  'assists': 7},
 {'match_id': 6983215361,
  'start_time': 1674457386,
  'duration': 1663,
  'radiant_win': True,
  'leagueid': 14960,
  'league_name': 'Golden Age',
  'radiant': False,
  'player_slot': 132,
  'account_id': 170267293,
  'kills': 4,
  'deaths': 7,
  'assists': 2},
 {'match_id': 6982393921,
  'start_time': 1674403004,
  'duration': 1799,
  'radiant_win': True,
  'leagueid': 12903,
  'league_name': 'YouTube SIVVIT - Big Russian Show',
  'radiant': True,
  'player_slot': 2,
  'account_id': 371293069,
  'kills': 17,
  'deaths': 1,
  'assists': 7},
 {'match_id': 6982133532,
  'start_time': 1674393982,
  'duration': 2002,
  'radiant_win': True,
  'leagueid': 14894,
  'league_name': 'Ancients League',
  'radiant': False,
  'player_s

Get Pro Matches


In [11]:
pro_matches = []

In [14]:
for i in range(0, 10000, 1000):
    result = use_api(
        "https://api.opendota.com/api/explorer",
        params={"sql":
                    """
                        SELECT
                            matches.match_id
                        FROM
                            matches
                            JOIN match_patch USING ( match_id )
                            JOIN leagues USING ( leagueid )
                            JOIN player_matches USING ( match_id )
                            JOIN heroes ON heroes.id = player_matches.hero_id
                            LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
                            LEFT JOIN teams USING ( team_id )
                        WHERE
                        TRUE
                            AND match_patch.patch >= '7.28'
                            AND leagues.tier = 'professional'
                        GROUP BY
                            matches.match_id
                        ORDER BY
                            matches.match_id NULLS LAST
                            LIMIT 1000 OFFSET {}
                    """.format(i)}
    )
    pro_matches += result["rows"]
    print("First {} data fetched.".format(i + 1000))
    time.sleep(1)

First 1000 data fetched.
First 2000 data fetched.
First 3000 data fetched.
First 4000 data fetched.
First 5000 data fetched.
First 6000 data fetched.
First 7000 data fetched.
First 8000 data fetched.
First 9000 data fetched.
First 10000 data fetched.


In [15]:
pro_matches = [i["match_id"] for i in pro_matches]

In [25]:
data = []

In [26]:
def get_match_data(match_id):
    kept_keys = ["match_id", "radiant_win", "region", "patch", "picks_bans"]
    try:
        result = use_api("https://api.opendota.com/api/matches/{match_id}".format(match_id=match_id))
        data.append({k: result[k] for k in kept_keys})
        time.sleep(1)
    except Exception:
        pass

In [27]:
pool = Pool(5)
pool.map(get_match_data, pro_matches)
len(data)

9383

In [28]:
pd.DataFrame(data[:60])

,match_id,radiant_win,region,patch,picks_bans
0,5746455668,False,3,47,"[{'is_pick': False, 'hero_id': 85, 'team': 1, ..."
1,5795993534,True,3,47,"[{'is_pick': False, 'hero_id': 65, 'team': 0, ..."
2,5883814830,True,5,47,"[{'is_pick': False, 'hero_id': 102, 'team': 1,..."
3,5860105982,True,17,47,"[{'is_pick': False, 'hero_id': 38, 'team': 0, ..."
4,5831784791,True,3,47,"[{'is_pick': False, 'hero_id': 7, 'team': 1, '..."
5,5746480618,False,15,47,"[{'is_pick': False, 'hero_id': 98, 'team': 0, ..."
6,5796057679,False,3,47,"[{'is_pick': False, 'hero_id': 98, 'team': 1, ..."
7,5883815426,False,5,47,"[{'is_pick': False, 'hero_id': 91, 'team': 0, ..."
8,5860179780,False,17,47,"[{'is_pick': False, 'hero_id': 120, 'team': 1,..."
9,5832007409,False,17,47,"[{'is_pick': False, 'hero_id': 19, 'team': 1, ..."


In [29]:
with open("../data/raw_data/data.json", "w") as f:
    json.dump({"data": data}, f)